In [3]:
pip install segmentation_models_pytorch


  Using cached segmentation_models_pytorch-0.5.0-py3-none-any.whl.metadata (17 kB)
  Using cached huggingface_hub-0.33.2-py3-none-any.whl.metadata (14 kB)
  Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached timm-1.0.16-py3-none-any.whl.metadata (57 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached hf_xet-1.1.5-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (879 bytes)
Using cached segmentation_models_pytorch-0.5.0-py3-none-any.whl (154 kB)
Using cached huggingface_hub-0.33.2-py3-none-any.whl (515 kB)
Using cached hf_xet-1.1.5-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 1.5 MB/s eta 0:00:0000:0100:010m
Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (471 kB)
Using cached timm-1.0.16-py3-none-any.whl (2.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.

In [5]:
import torch
import segmentation_models_pytorch as smp

# Match the original training config: in_channels=3
model = smp.Unet(encoder_name="resnet34", in_channels=3, classes=1)
model.load_state_dict(torch.load("../models/unet-binary-masking.pth", map_location="cpu"))
model.eval()

# Dummy input must also have 3 channels
dummy_input = torch.randn(1, 3, 256, 256)

# Export to ONNX
torch.onnx.export(
    model,
    dummy_input,
    "../models/unet.onnx",
    input_names=["input"],
    output_names=["output"],
    opset_version=11
)


In [ ]:
pip uninstall tensorflow-probability -y
pip install tensorflow-probability==0.20.0


SyntaxError: invalid syntax (2852171440.py, line 1)

In [5]:
from onnx_tf.backend import prepare
import onnx

# Load the ONNX model
onnx_model = onnx.load("../models/unet.onnx")

# Convert ONNX to TensorFlow SavedModel
tf_rep = prepare(onnx_model)
tf_rep.export_graph("../models/unet_tf")


/home/abdullah-bin-mansoor/miniconda3/envs/py310/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/abdullah-bin-mansoor/miniconda3/envs/py310/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were

INFO:tensorflow:Assets written to: ../models/unet_tf/assets


INFO:tensorflow:Assets written to: ../models/unet_tf/assets


In [6]:
import tensorflow as tf

# Load the SavedModel directory
saved_model_dir = "../models/unet_tf"  # or wherever it was saved

# Initialize converter
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

# Optional: Enable optimization (smaller model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert the model
tflite_model = converter.convert()

# Save the .tflite model
with open("../models/unet.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ TFLite model saved to ../models/unet.tflite")


2025-07-08 15:58:32.872561: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-07-08 15:58:32.872639: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-07-08 15:58:32.888519: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ../models/unet_tf
2025-07-08 15:58:32.934980: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2025-07-08 15:58:32.935022: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ../models/unet_tf
2025-07-08 15:58:33.182622: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2025-07-08 15:58:33.208145: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-07-08 15:58:33.744778: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: ../models/unet_tf
2025

✅ TFLite model saved to ../models/unet.tflite


In [2]:
# save_as_tflite.py
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model('./models/yolo_tf')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('./models/yolo-segmentation-weights.tflite', 'wb') as f:
    f.write(tflite_model)


2025-07-10 12:41:02.809355: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-10 12:41:07.851929: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-10 12:41:07.907676: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/abdullah-bin-mansoor/miniconda3/envs/py310/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/abdullah-bin-mansoor/miniconda3/envs/py310/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/abdullah-bin-mansoor/miniconda3/envs/py310/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/abdullah-bin-mansoor/miniconda3/envs/py310/lib/python3.10

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

...


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/abdullah-bin-mansoor/miniconda3/envs/py310/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/abdullah-bin-mansoor/miniconda3/envs/py310/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/abdullah-bin-mansoor/miniconda3/envs/py310/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/abdullah-bin-mansoor/miniconda3/envs/py310/lib/python3.10

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle